In [1]:
##The feature engineering was for trying on featuretools DSF method 

In [2]:
import featuretools as ft
import numpy as np
import pandas as pd

In [19]:
#df_trans = pd.read_csv('train_transaction.csv')
df_trans = pd.read_csv('train_transaction.csv',index_col='TransactionID')
df_test_trans = pd.read_csv('test_transaction.csv',index_col='TransactionID')

#df_id = pd.read_csv('train_identity.csv')
df_id = pd.read_csv('train_identity.csv',index_col='TransactionID')
df_test_id = pd.read_csv('test_identity.csv',index_col='TransactionID')

##make the data frame shorter for space usage


In [20]:
##merge the data sets 
df_train = df_trans.merge(df_id, how='left', left_index=True,  on='TransactionID')
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True, on='TransactionID')

In [21]:
##save out ids and target
trans_ID = df_train.index
fraud = df_train['isFraud']
df_train.drop(['isFraud'], axis = 1, inplace = True)
print(df_train)

MemoryError: Unable to allocate array with shape (399, 590540) and data type float64

In [ ]:
##combine the train and test data sets 
combi = df_train.append(df_test, ignore_index = True)


In [ ]:
print(df_train.head())

print(combi)

In [ ]:
combi.isnull().sum()


In [9]:
df_train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [ ]:
from sklearn import preprocessing

#label encoding the rest 
for col in combi.columns:
    if combi[col].dtype == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(list(combi[col].astype(str).values))
        combi[col] = le.transform(list(combi[col].astype(str).values))

#fill missing values
combi.fillna(-999,inplace = True)

In [ ]:
# drop all V features
V = []
for col in combi.columns:
    if 'V' in col:
        V.append(col)
print(V)        

combi.drop(V,axis = 1, inplace= True)


In [ ]:
# creating and entity set 

es = ft.EntitySet(id = 'fraud')

# adding a dataframe

es.entity_from_dataframe(entity_id = 'trans', dataframe = combi, index = 'id')

In [13]:
print(es)

Entityset: fraud
  Entities:
    trans [Rows: 2000, Columns: 94]
  Relationships:
    No relationships


In [ ]:
##The following codes split transaction dataset into two entities: trans without all idxx columns and 
##id with all idxx columns  


#just pick three variables as the 2nd depth 
es.normalize_entity(base_entity_id='trans', new_entity_id='id', index = 'id_01', 
                    additional_variables = ['id_11'])



In [ ]:
##show the dataframe in the generated entity id. 
##every row represent each unique id_01, and all non-index columns are the values that firstly occur. 
##only the index will be used as the factor for aggregration and transformation of the target entity.
print(combi.loc[5:50,'id_01':'id_02'])
print(es['id'].df['id_11'])
es['trans'].df

In [3]:
##let us look at all the feature primitives in Featuretools:
## it contains types of aggregation and transform 

pd.options.display.max_rows = 100
ft.list_primitives()

,name,type,description
0,max,aggregation,"Calculates the highest value, ignoring `NaN` v..."
1,time_since_first,aggregation,Calculates the time elapsed since the first da...
2,all,aggregation,Calculates if all values are 'True' in a list.
3,skew,aggregation,Computes the extent to which a distribution di...
4,mode,aggregation,Determines the most commonly repeated value.
5,min,aggregation,"Calculates the smallest value, ignoring `NaN` ..."
6,percent_true,aggregation,Determines the percent of `True` values.
7,any,aggregation,Determines if any value is 'True' in a list.
8,last,aggregation,Determines the last value in a list.
9,count,aggregation,"Determines the total number of values, excludi..."


In [ ]:
##Here we create new features using specified primitives.
##The target entity is the trans dataframe 

features, feature_names = ft.dfs(entityset = es, target_entity ='trans',
                                agg_primitives = ["mean","max","median"])

##new data frame with newly created features. 
print(feature_names)
features.head()

In [ ]:
##This is the 2nd way to generate in-depth features using the two entities
features_2dep, feature_names_2dep = ft.dfs(entityset = es, target_entity ='trans', max_depth = 2, verbose = 1, n_jobs = 3)

print(feature_names_2dep)
features_2dep.head()

In [ ]:
###interpretation:
###id.MEAN(trans.M4) means the mean value that the M4 feature in transaction data frame 
